In [12]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import scipy.sparse
from fastFM import sgd

In [3]:
train_set = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f2/train_scratch.csv', delimiter = '\t')

In [4]:
val_set = pd.read_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f2/val_scratch.csv', delimiter = '\t')

In [8]:
train_set.columns

Index(['engaged_user_id', 'engaging_user_id', 'retweet_with_comment_bool',
       'day_of_tweet'],
      dtype='object')

In [9]:
train_set_dictionary = train_set[['engaged_user_id', 'engaging_user_id', 'day_of_tweet']].to_dict('records')

In [10]:
val_set_dictionary = val_set[['engaged_user_id', 'engaging_user_id', 'day_of_tweet']].to_dict('records')

In [13]:
v = DictVectorizer(sparse=True)

In [14]:
X = v.fit_transform(train_set_dictionary)

In [15]:
X_val = v.transform(val_set_dictionary)

In [16]:
y = train_set[['retweet_with_comment_bool']].values.squeeze()

In [17]:
y_gt = val_set[['retweet_with_comment_bool']].values.squeeze()

In [18]:
y[y == 0] = -1

In [25]:
fm = sgd.FMClassification(n_iter=1000, init_stdev=0.1, rank=16, l2_reg_w=0.3, l2_reg_V=0.5, step_size = 0.001)

In [26]:
fm.fit(X, y)

FMClassification(init_stdev=0.1, l2_reg=None, l2_reg_V=0.5, l2_reg_w=0.3,
                 n_iter=1000, random_state=123, rank=16, step_size=0.001)

In [27]:
y_predicted = fm.predict_proba(X_val)

In [22]:
from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [28]:
compute_prauc(y_predicted, y_gt)

0.008021518299717336

In [29]:
compute_rce(y_predicted, y_gt)

-268.911005677649